In [1]:
var avro = require('avsc');

const snappy = require('snappy'); // Or your favorite Snappy library.
const codecs = {
  snappy: function (buf, cb) {
    // Avro appends checksums to compressed blocks, which we skip here.
    return snappy.uncompress(buf.slice(0, buf.length - 4), cb);
  }
};

const testFolder = './trades/';
const fs = require('fs');

In [4]:
var messages = []
fs.readdir(testFolder, (err, files) => {
  files.forEach(file => {
    avro.createFileDecoder(testFolder+file, {codecs})
  .on('metadata', function (type) { })
  .on('data', function (val) { messages.push(val) });
  });
});


// Or, if you just want the file's header (which includes the schema):
// var header = avro.extractFileHeader('./trades.avro');

In [5]:
var msgs = messages.filter(function(msg) {
    let d = JSON.parse(msg.message.toString('utf8'))
    return (d['exchange'] === "okex_spot" && d['symbol'] === "ETHBTC")
})

In [6]:
msgs.length

11828

In [7]:
msgs[0].message.toString('utf-8')

'{"event_id": ["okex_spot", "trade", "ETH", "BTC", "1558033200379"], "event_time": 1558033200.379, "event_datetime": "2019-05-16T19:00:00+0000", "event_type": "trade", "event_time_ms": 1558033200379, "produced_time": 1558033200.854859, "produced_datetime": "2019-05-16T19:00:00", "exchange": "okex_spot", "symbol": "ETHBTC", "quote_asset": "BTC", "base_asset": "ETH", "market_type": "spot", "trade_time": null, "quantity": 4.40583, "price": 0.03327, "side": "buy", "is_maker": null, "order_type": null, "trade_id": 662380915, "__faust": {"ns": "models.trade.Trade"}}'

In [8]:
const map1 = msgs
    .map(x => JSON.parse(x.message.toString('utf-8')))
    .map(function(x){
        return {
        'time':x["event_time_ms"],
        'side':x["side"],
        'quantity':x["quantity"],
        'price': x["price"]
        }
    })

In [10]:
fs.writeFile('eth_btc_okex_trades.json', JSON.stringify(map1), 'utf8', function(err){if(err){throw err}});